In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWa

In [2]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

#输入的图片是28*28
n_inputs=28 #输入一行，一行有28个数据,
max_time=28 #一共28行，相当于序列，一个图片有28个序列，一个样本输入28次
lstm_size=100 #隐层单元，比一般神经元要复杂的神经元
n_classes=10 #10个分类
batch_size=50 #每批次50个样本
n_batch=mnist.train.num_examples // batch_size #计算一共有多少批次

#这里的none表示第一维度可以是任意的长度
x=tf.placeholder(tf.float32,[None,784])
#正确的标签
y=tf.placeholder(tf.float32,[None,10])

#初始化权值
weights=tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
#初始化偏执值
biases=tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义RNN网络
def RNN(X,weight,biases):
    inputs=tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(lstm_size)#BasicLSTMCell是基本LSTN单元
    #final_state[state,batch_size,cell.state_size]，这里state为0或1，cell.state_size为lstm_size
    #final_state[0]是cell state
    #final_state[1]是hidden state
    #output:The rnn output 'Tensor'  （随max_time值变化而记录每一次outputs）
    #    if time_major==False(default),this will be a 'Tensor' shaped:
    #       '[batch_size,max_time,cell.output_size]'
    #    if time_major==Ture,this will be a 'Tensor' shaped:
    #       '[max_time,batch_size,cell.output_size]'  cell.output_size和cell.state_size一样
    outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results=tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)#最终输出，输入一张图片的最终输出，用outputs最后的值也可以
    return results

#计算rnn的返回结果
prediction=RNN(x, weights, biases)
#损失函数
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
#使用AdamOptimizer进行优化
trian_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) #argmax返回一维张量中最大的值所在的位置
#求准确率
accuarcy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #把correct_prediction变为float32类型
#初始化
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(trian_step, feed_dict={x:batch_xs,y:batch_ys})

        acc=sess.run(accuarcy, feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print ("Iter "+str(epoch)+", Testing Accuarcy= " + str(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter 0, Testing Accuarcy= 0.7501
Iter 1, Testing Accuarcy= 0.8081
Iter 2, Testing Accuarcy= 0.8299
Iter 3, Te